In [1]:
# Importing basic libraries
import pandas as pd
import numpy as np

# for graphing
import matplotlib.pyplot as plt
import seaborn as sns

# regression libraries will be imported in the process to maintain an overview of when we need what

### Introduction
1. We have a target, therefore we use supervised learning. 
2. As we are seeking out a categorical value as a target (intiger and categroical scale), we cannot perform a regression analysis. We will go with a classification model
3. We know about and feel comfortable with the following classification models: <br>
    3.1. Decision Tree <br>
    3.2. Logisitc Regression** <br>
    3.3. KNN <br>


** Logisitc Regression is only used when we have one dependent variable and one independent variables. To carry this type of model out for multiple variables, we would need to do a mulitnominal or ordinal regression, however, we are not comfortable with these, so we will not attempt them on our first go. 

In [2]:
df = pd.read_csv("../Data/cleaned_cookie_data.csv", index_col=0)

In [3]:
df.head()

,sugar to flour ratio,sugar index,bake temp,chill time,calories,pH,grams baking soda,bake time,quality,butter type,weight,mixins,crunch factor
0,0.25,9.5,300,15.0,136.0,8.10,0.44,12.1,8,melted,15.2,raisins,1.30
1,0.23,3.3,520,34.0,113.0,8.16,0.48,8.4,7,melted,12.4,raisins,1.71
2,0.18,1.9,360,33.0,106.0,8.21,0.83,14.0,9,melted,9.4,"nuts, chocolate",1.78
3,0.18,10.5,490,41.0,124.0,8.14,0.35,10.5,7,melted,12.2,chocolate,1.59
4,0.24,2.4,770,6.0,33.0,8.09,0.57,9.4,5,cubed,19.8,"nuts, oats, chocolate",1.30


## Pre-processing
For preprocessing we will do the following:
- encoding categorical variables
- define X and y
- split into train and test (we chose a 80:20 ratio)

In [4]:
# Selecting data that are our features and the target variables.
y = df[['quality']]
X = df.drop(columns=['quality'])

##### Encoding, dummy variables for mixins

In [9]:
df["mixins"].value_counts().index

Index(['chocolate', 'raisins', 'chocolate, oats', 'nuts, chocolate',
       'nuts,raisins', 'nuts, oats, chocolate', 'nuts, oats',
       'chocolate, peanut butter', 'raisins, oats', 'peanut butter', 'oats',
       'chocolate, oats, peanut butter', 'No mixins'],
      dtype='object')

In [15]:
# create dummy variables
mixes = ['raisins', 'chocolate', 'oats', 'nuts', 'peanut butter', 'No mixins']

# Your code here
for i in mixes:
    df[i]=0

In [17]:
# fill dummy
for mix in mixes:
    
    df[mix] = df.mixins.str.contains(mix, regex=False).astype(int)

In [18]:
df.head()

,sugar to flour ratio,sugar index,bake temp,chill time,calories,pH,grams baking soda,bake time,quality,butter type,weight,mixins,crunch factor,raisins,chocolate,oats,nuts,peanut butter,No mixins
0,0.25,9.5,300,15.0,136.0,8.10,0.44,12.1,8,melted,15.2,raisins,1.30,1,0,0,0,0,0
1,0.23,3.3,520,34.0,113.0,8.16,0.48,8.4,7,melted,12.4,raisins,1.71,1,0,0,0,0,0
2,0.18,1.9,360,33.0,106.0,8.21,0.83,14.0,9,melted,9.4,"nuts, chocolate",1.78,0,1,0,1,0,0
3,0.18,10.5,490,41.0,124.0,8.14,0.35,10.5,7,melted,12.2,chocolate,1.59,0,1,0,0,0,0
4,0.24,2.4,770,6.0,33.0,8.09,0.57,9.4,5,cubed,19.8,"nuts, oats, chocolate",1.30,0,1,1,1,0,0


In [19]:
# drop mixins
df.drop("mixins", axis=1, inplace=True)

##### Encoding, dummy variables for butter

In [ ]:
# we can OneHotEncoder
from sklearn.preprocessing import OneHotEncoder


###### Splitting, fitting, training

In [5]:
# Importing sklearn library for model selection to split between train and test
from sklearn.model_selection import train_test_split

# Splitting data into train and test 
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=12)